In [1]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1' # to disable GPU
from ts_data.ts_data import ts_data as ts
import getStocks
import visualize
import ts_models.predicts
import pandas as pd
import ts_models.ts_lstm as ts_lstm

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

/home/ahendel/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# view tenslow enabled devices
#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

# User parameters

In [20]:
entityID = 'amd'.upper()
n_in = 360 # number of timesteps as input
n_out = 360 # number of timesteps as output
target = 'close_AMD'

# Load Data

In [16]:
dataset = pd.read_csv('/mnt/ubudata/projects/data/AMD_plus_100.csv')
# dataframe should be in INCREASING chronological order

In [17]:
# ie. row 1 is Jan 1, row 2 in Jan 2.
dataset.sort_values('dtg', inplace=True)
# and do a forward fill
dataset.fillna(method='ffill', inplace=True)
# then drop the NA
dataset.dropna(inplace=True)

In [27]:
# rename dtg to date for TS package
dataset.rename(columns={'dtg':'Date'}, inplace=True)

In [28]:
dataset.head()

,Date,close_AMD,volume_AMD,close_ANAT,volume_ANAT,close_NTRA,volume_NTRA,close_KMI,volume_KMI,close_IROQ,volume_IROQ,close_CCD,volume_CCD,close_FAD,volume_FAD,close_IXY,volume_IXY,close_FBIZ,volume_FBIZ,close_INDY,volume_INDY,close_CCLP,volume_CCLP,close_CPLP,volume_CPLP,close_ROBT,volume_ROBT,close_CNSL,volume_CNSL,close_MNST,volume_MNST,close_FINX,volume_FINX,close_DNLI,volume_DNLI,close_CARS,volume_CARS,close_KTOV,volume_KTOV,close_MARA,volume_MARA,close_GSBC,volume_GSBC,close_RFEM,volume_RFEM,close_CODA,volume_CODA,close_ARRS,volume_ARRS,close_SPA,volume_SPA,close_TC,volume_TC,close_ETX,volume_ETX,close_PVTL,volume_PVTL,close_DOVA,volume_DOVA,close_OCSL,volume_OCSL,close_AGS,volume_AGS,close_CYAD,volume_CYAD,close_AMRB,volume_AMRB,close_FORR,volume_FORR,close_PFIN,volume_PFIN,close_HMI,volume_HMI,close_TUSA,volume_TUSA,close_RBBN,volume_RBBN,close_CERC,volume_CERC,close_FEMB,volume_FEMB,close_CCCR,volume_CCCR,close_CUTR,volume_CUTR,close_CENT,volume_CENT,close_KERX,volume_KERX,close_IIJI,volume_IIJI,close_HTHT,volume_HTHT,close_JCS,volume_JCS,close_ILMN,volume_ILMN,close_IFEU,volume_IFEU,close_BIIB,volume_BIIB,close_DTYS,volume_DTYS,close_CREE,volume_CREE,close_CARA,volume_CARA,close_SCON,volume_SCON,close_LMRKN,volume_LMRKN,close_ENTG,volume_ENTG,close_CHKP,volume_CHKP,close_QRHC,volume_QRHC,close_WIRE,volume_WIRE,close_ITCI,volume_ITCI,close_FULT,volume_FULT,close_JNCE,volume_JNCE,close_FPA,volume_FPA,close_IAC,volume_IAC,close_CRBP,volume_CRBP,close_NANO,volume_NANO,close_NAVI,volume_NAVI,close_LKM,volume_LKM,close_GLAD,volume_GLAD,close_IRDM,volume_IRDM,close_HYGS,volume_HYGS,close_CTIC,volume_CTIC,close_FIN,volume_FIN,close_NEON,volume_NEON,close_FNGN,volume_FNGN,close_FEX,volume_FEX,close_MZOR,volume_MZOR,close_FTGC,volume_FTGC,close_FISV,volume_FISV,close_KIN,volume_KIN,close_PXLW,volume_PXLW,close_ADTN,volume_ADTN,close_LHCG,volume_LHCG,close_AVXS,volume_AVXS,close_ANSS,volume_ANSS,close_FMK,volume_FMK,close_CASM,volume_CASM,close_ATLC,volume_ATLC,close_HMNF,volume_HMNF,close_UHAL,volume_UHAL,close_GTIM,volume_GTIM,close_ANTH,volume_ANTH,close_CVV,volume_CVV,close_ISNS,volume_ISNS,close_ZUO,volume_ZUO,close_IRWD,volume_IRWD,close_KANG,volume_KANG,close_FCVT,volume_FCVT,close_ALTR,volume_ALTR,close_KLXI,volume_KLXI,close_CHSCN,volume_CHSCN
163826,2018-06-08 11:46:00,15.1700,148004.0,121.78,4354.0,11.01,4814.0,16.26,259725.0,20.0,5.0,20.4501,1652.0,67.61,954.0,2.55,600.0,25.3,1132.0,35.23,13108.0,7.41,522.0,3.24,10786.0,28.75,200.0,11.23,51566.0,54.41,234034.0,23.16,320.0,18.36,5344.0,27.40,3200.0,2.46,2749.0,1.76,33157.0,53.85,2689.0,71.6696,250.0,3.6,65.0,26.345,1211.0,18.215,100.0,86.46,58836.0,5.08,15547.0,21.2200,9400.0,24.51,3890.0,4.32,4496.0,29.84,95240.0,29.99,100.0,15.28,248.0,45.5,2837.0,8.75,291.0,12.00,3700.0,32.94,100.0,6.155,500.0,4.27,4376.0,41.41,695.0,1.1101,780.0,51.0,9126.0,38.88,2534.0,4.61,81802.0,10.2,2426.0,132.16,9075.0,3.53,278.0,231.19,82454.0,41.02,550.0,269.45,172690.0,24.097,1767.0,48.95,500.0,12.48,47744.0,0.9798,128.0,23.73,200.0,33.3,123438.0,99.0000,500.0,2.0099,9793.0,55.0,7891.0,18.16,22914.0,17.3,77181.0,19.29,5734.0,35.04,601.0,157.43,18419.0,6.3,13542.0,25.48,9853.0,13.26,225096.0,1.101,700.0,8.61,1760.0,12.45,103164.0,8.25,463.0,3.9,4065.0,8.65,100.0,0.439,130.0,35.55,24386.0,58.51,200.0,52.98,12671.0,21.27,828.0,75.19,600.0,8.85,9098.0,4.13,100.0,14.7,55549.0,75.59,10893.0,212.48,16010.0,171.23,2100.0,33.1599,147.0,1.17,607.0,2.12,917.0,18.75,300.0,345.55,8980.0,3.25,200.0,0.3223,1239.0,7.89,200.0,4.45,783.0,29.3198,600.0,16.95,233848.0,20.0,18824.0,28.993,204.0,35.720,2100.0,74.47,26718.0,27.87,1437.0
166120,2018-06-08 11:47:00,15.1748,153207.0,121.78,4354.0,11.01,4814.0,16.26,259725.0,20.0,5.0,20.4501,1652.0,67.61,954.0,2.55,600.0,25.3,1132.0,35.23,13108.0,7.41,522.0,3.24,10786.0,28.75,200.0,11.23,51566.0,54.41,234034.0,23.16,320.0,18.36,5344.0,27.46,2444.0,2.46,2749.0,1.76,33157.0,53.85,2689.0,71.6696,250.0,3.6,65.0,26.330,300.0,18.240,1400.0,86.46,58836.0,5.08,15547.0,21.2

In [29]:
dataset.shape

(2541, 203)

## Pick the Sector and/or Industry

Purpose is to come up with a list of tickers and their corresponding exchange

In [ ]:
#d=getStocks.get_tickers_industry()
#d.to_csv('tickers_w_industry.csv')

In [21]:
import imp
imp.reload(getStocks)

<module 'getStocks' from '/mnt/ubudata/projects/forecast-engine/src/getStocks.py'>

In [13]:
# load dataset
# dataset = getStocks.get_single(ticker=ticker, save=True)
# dataset = getStocks.load_single(ticker)
# dataset = getStocks.join_tgt_spt(target_ticker=entityID.upper(), industry=['Semiconductors'], exclude=['MU','QRVO'])
# dataset.reset_index(level=0, inplace=True, drop=True)
# dataset.rename(columns=lambda x: x.replace(' ',''),inplace=True)

# Prepare Data for LSTM

In [30]:
ts_data = ts(n_in=n_in, 
             n_out=n_out, 
             entityID=entityID,
             target=target,
             rawData=dataset)

In [31]:
ts_data.eng_features(derivate=False, weekdays=False)

In [ ]:
ts_data.roll_data()

In [ ]:
ts_data.data.to_csv('AMD_TS_DATA.csv',index=None)

In [ ]:
ts_data.tscv(train=0.98)

In [ ]:
ts_data.data.tail()

# Build LSTM Architecture

In [ ]:
ts_model = ts_lstm.lstm_model(ts_data, 
                               inlayer=int(ts_data.train_X.shape[-1])*2,
                               hiddenlayers=0,
                               loss_function='mae',
                               dropout=0.05,
                               activation='tanh',
                               gpus=1)

# Fit model

In [ ]:
import time
start = time.time()
history = ts_model.fit(ts_data.train_X, ts_data.train_y, 
                    epochs=50, 
                    batch_size=1024, 
                    validation_data=(ts_data.test_X, ts_data.test_y), 
                    verbose=2, 
                    shuffle=False)
fitTime = time.time()-start
print('Fit Time: {}'.format(round(fitTime,2)))

In [ ]:
visualize.plot_loss(history)

# Results

In [ ]:
# select the last sequence of data (it is length of n_in)
test_X = ts_data.test_X[-1]
test_Y = ts_data.test_y[-1]

In [ ]:
test_X.reshape(1, 100, 70).shape

In [ ]:
yhat = ts_model.predict(test_X.reshape(1, test_X.shape[-2], test_X.shape[-1]))

In [ ]:
import matplotlib.pyplot as plt
x = [x for x in range(len(yhat[0]))]
plt.plot(x, yhat[0], label='predicted change in price')
plt.plot(x, test_Y, label='actual change in price')
plt.title(entityID)
plt.xlabel('days')
plt.ylabel('amount change in price')
plt.legend()
plt.show()